# Importing the Libraries

In [1]:
import os
from shutil import copyfile
import pandas as pd
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout

C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\User\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (

# Data Path

In [39]:
train_directory='C:/Users/User/Python/Projects/Distracted Driver Detection/Data/train/'
test_directory='C:/Users/User/Python/Projects/Distracted Driver Detection/Data/test/'

# For training the model, I am first dividing the Training Data on 8:2 ratio.
# Training the model on this 80% Data at first and then checking the accuracy on the remaining 20% by comparing it with its actual output.

Creating two separate folders for Training and Testing purposes having the same folder structure as Input Training Data and then copying the files in 8:2 ratio.

In [15]:
c=0
data={}
# os.listdir() method in python is used to get the list of all files and directories in the specified directory. 
for (filenames) in os.listdir(train_directory):

    for imgfile in os.listdir(train_directory+ filenames):

        if filenames in data:
            data[filenames].append(imgfile)
        else:
            data[filenames]=[imgfile]

print("Category Names:",end=" ")
for i in data:
    print(i,end=" ")

Category Names: c0 c1 c2 c3 c4 c5 c6 c7 c8 c9 

In [6]:
# creating separate folders so that training data can be divided for model training
categories_list=list(data.keys())
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')
for category in categories_list:
    os.mkdir(os.path.join('master_data/training/',category))
    os.mkdir(os.path.join('master_data/testing/',category))

In [7]:
# Keeping 80% training data in master_data/training and 20% in master_data/testing.
# This 20% data will be later used to check my model accuracy.
split_size=0.8
for category,images in data.items():
    train_size=int(split_size*len(images))
    train_images=images[:train_size]
    test_images=images[train_size:]
    for image in train_images:
        source=os.path.join('Data/train',category,image)
        dest=os.path.join('Code/master_data/training',category,image)
        copyfile(source,dest)
    for image in test_images:
        source=os.path.join('Data/train',category,image)
        dest=os.path.join('Code/master_data/testing',category,image)
        copyfile(source,dest)

# CNN Model

In [18]:
#creating the CNN Model.
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(100,100,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 100, 100, 64)      832       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 50, 50, 128)       32896     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 25, 25, 256)       131328    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 12, 12, 512)      

# Fitting and computing the values of the parameters of the CNN model

In [19]:
# Giving the directory paths to Image Data Generator.
train_dir='master_data/training'
test_dir='master_data/testing'
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(100,100),class_mode='categorical',batch_size=100)
test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(test_dir,target_size=(100,100),class_mode='categorical',batch_size=100)

es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.02)

Found 17934 images belonging to 10 classes.
Found 4490 images belonging to 10 classes.


In [20]:
# 80% of Training data is fitted. We can see the accuracy for testing data as it is provided in validation_data parameter
model.fit_generator(train_generator,validation_data=test_generator,epochs=5,verbose=1,callbacks=[es])

Epoch 1/5
180/180 [==============================] - 325s 2s/step - loss: 1.2140 - acc: 0.5686 - val_loss: 0.2528 - val_acc: 0.9298
Epoch 2/5
180/180 [==============================] - 326s 2s/step - loss: 0.2785 - acc: 0.9142 - val_loss: 0.1117 - val_acc: 0.9690
Epoch 3/5
180/180 [==============================] - 327s 2s/step - loss: 0.1489 - acc: 0.9553 - val_loss: 0.0626 - val_acc: 0.9833
Epoch 4/5
180/180 [==============================] - 345s 2s/step - loss: 0.1026 - acc: 0.9695 - val_loss: 0.0525 - val_acc: 0.9864


# Checking the accuracy to determine model performance

In [21]:
pred=model.predict_generator(test_generator,verbose=1)

45/45 [==============================] - 31s 694ms/step


In [26]:
# Getting the loss and accuracy on the 20% training data which we kept for testing purposes
loss,accuracy=model.evaluate_generator(test_generator)
print("Loss: ",loss)
print("Accuracy: ",accuracy)

Loss:  0.05249389924315943
Accuracy:  0.98641425


# For Testing Data need to fit the whole Training Data now

In [27]:
train_datagen_complete=ImageDataGenerator(rescale=1.0/255)
train_generator_complete=train_datagen_complete.flow_from_directory(train_directory,target_size=(100,100),class_mode='categorical',batch_size=100)
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.02)


Found 22424 images belonging to 10 classes.


Reinitializing model

In [28]:
# need to reinitialize the model before the whole training data fitting
model = Sequential()

model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(100,100,3)))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=256, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(filters=512, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 100, 100, 64)      832       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 50, 50, 128)       32896     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 25, 25, 128)       0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 25, 25, 256)       131328    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 12, 12, 256)       0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 12, 12, 512)      

In [29]:
#fitting the whole training data on the CNN model
model.fit_generator(train_generator_complete,epochs=5,verbose=1,callbacks=[es])

Epoch 1/5
225/225 [==============================] - 548s 2s/step - loss: 1.1583 - acc: 0.6038
Epoch 2/5
225/225 [==============================] - 370s 2s/step - loss: 0.2003 - acc: 0.9403
Epoch 3/5
225/225 [==============================] - 372s 2s/step - loss: 0.1055 - acc: 0.9691
Epoch 4/5
225/225 [==============================] - 372s 2s/step - loss: 0.0696 - acc: 0.9789
Epoch 5/5
225/225 [==============================] - 371s 2s/step - loss: 0.0577 - acc: 0.9837


# The folder structure given for the Testing Data is not compatible with Image Data Generator. Need to create a new folder structure so that Data can be fed to the Image Data Generator.  

In [41]:
test_images=[]
for (filenames) in os.listdir(test_directory):
    test_images.append(filenames)    

In [45]:
os.mkdir('master_data/Unknown_Image_Testing')
os.mkdir('master_data/Unknown_Image_Testing/Images')

for image in test_images:
    source=os.path.join(test_directory,image)
    dest=os.path.join('master_data/Unknown_Image_Testing/Images/',image)    
    copyfile(source,dest)  

In [46]:
test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator = test_datagen.flow_from_directory(
    directory='master_data/Unknown_Image_Testing/',
    target_size=(100,100),
    batch_size=100,
    shuffle=False
)

Found 79726 images belonging to 1 classes.


# Predicting the output for the images of the Drivers

In [47]:
# Predicting the output for the testing data i.e whether the drivers were distracted or not.
pred=model.predict_generator(test_generator,verbose=1)

798/798 [==============================] - 627s 785ms/step


In [48]:
print(pred.shape)

(79726, 10)


# Storing the output in a dataframe

In [53]:
df = pd.DataFrame({'img':[],'SAFE_DRIVING':[], 'TEXTING_RIGHT':[],'TALKING_PHONE_RIGHT':[], 
                   'TEXTING_LEFT':[], 'TALKING_PHONE_LEFT':[],'OPERATING_RADIO':[], 'DRINKING':[], 
                   'REACHING_BEHIND':[], 'HAIR_AND_MAKEUP':[], 'TALKING_TO_PASSENGER':[]})
df

,img,SAFE_DRIVING,TEXTING_RIGHT,TALKING_PHONE_RIGHT,TEXTING_LEFT,TALKING_PHONE_LEFT,OPERATING_RADIO,DRINKING,REACHING_BEHIND,HAIR_AND_MAKEUP,TALKING_TO_PASSENGER


In [54]:
c=0
for i in pred:
    df = df.append(  #Append rows of other to the end of caller, returning a new object.
            {
                'img':test_images[c],
                'SAFE_DRIVING':round(i[0],2), 
                'TEXTING_RIGHT':round(i[1],2),
                'TALKING_PHONE_RIGHT':round(i[2],2),
                'TEXTING_LEFT':round(i[3],2),
                'TALKING_PHONE_LEFT':round(i[4],2),
                'OPERATING_RADIO':round(i[5],2),
                'DRINKING':round(i[6],2),
                'REACHING_BEHIND':round(i[7],2),
                'HAIR_AND_MAKEUP':round(i[8],2),
                'TALKING_TO_PASSENGER':round(i[9],2)
                
            },
            ignore_index=True  #If True, the resulting axis will be labeled 0, 1, …, n - 1.
              )
    c+=1

In [55]:
df

,img,SAFE_DRIVING,TEXTING_RIGHT,TALKING_PHONE_RIGHT,TEXTING_LEFT,TALKING_PHONE_LEFT,OPERATING_RADIO,DRINKING,REACHING_BEHIND,HAIR_AND_MAKEUP,TALKING_TO_PASSENGER
0,img_1.jpg,0.00,0.00,0.00,0.0,0.00,1.0,0.00,0.00,0.00,0.00
1,img_10.jpg,0.00,0.00,0.00,0.0,0.00,1.0,0.00,0.00,0.00,0.00
2,img_100.jpg,0.55,0.15,0.00,0.0,0.00,0.0,0.00,0.00,0.00,0.29
3,img_1000.jpg,0.00,0.00,0.00,0.0,0.00,0.0,0.08,0.00,0.92,0.00
4,img_100000.jpg,0.00,0.00,0.00,0.0,0.99,0.0,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...
79721,img_99994.jpg,0.00,0.12,0.15,0.0,0.00,0.0,0.00,0.57,0.15,0.00
79722,img_99995.jpg,0.00,0.00,0.00,1.0,0.00,0.0,0.00,0.00,0.00,0.00
79723,img_99996.jpg,0.00,0.00,0.00,0.0,0.99,0.0,0.01,0.00,0.01,0.00
79724,img_99998.jpg,0.00,0.00,0.00,0.0,0.00,0.0,1.00,0.00,0.00,0.00


# Finally storing the output in a CSV file

In [56]:
df.to_csv('Test_Data_Output.csv',index = False)